In [6]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import datetime
import time
import os

In [7]:
# data dictionary
'''
    用户标识（uId）匿名化处理后的用户唯一标识（ID取值从1000001开始，依次递增）
    应用标识（appId）匿名化处理后的app唯一标识
    使用时长（duration）1天内用户对某app的累计使用时长
    打开次数（times）1天内用户对某app的累计打开次数
    使用日期（use_date）用户对某app的使用日期
'''

'\n    用户标识（uId）匿名化处理后的用户唯一标识（ID取值从1000001开始，依次递增）\n    应用标识（appId）匿名化处理后的app唯一标识\n    使用时长（duration）1天内用户对某app的累计使用时长\n    打开次数（times）1天内用户对某app的累计打开次数\n    使用日期（use_date）用户对某app的使用日期\n'

In [8]:
## data preprocessing
### filename : user_app_usage.csv
### assumptions:


In [9]:
# function blocks:
def dateToInt(minDate='2019-02-01',maxDate='2019-04-30'):
    '''
    for cleaning the datetime-type data
    start from 0 increasingly by 1
    '''
    d = {}
    i = 1
    start = datetime.datetime.strptime(minDate,'%Y-%m-%d')
    end = datetime.datetime.strptime(maxDate,'%Y-%m-%d')
    time = start
    while time <= end:
        d[datetime.datetime.strftime(time,'%Y-%m-%d')]=i
        time = time + datetime.timedelta(1)
        i+=1
#     print(np.array([str(v)+' : '+str(k) for k,v in d.items()],dtype='str').reshape(-1,1)) # meanning of integer
    return d


def appIdToInt(ls):
    '''
    for cleaning the uId
    translate the uId to integer
    '''
    d={}
    i=1
    for v in ls:
        if v in d:
            pass
        else:
            d[v]=i
            i+=1
#     print(np.array([str(v)+' : '+str(k) for k,v in d.items()],dtype='str').reshape(-1,1))
    return d

def fileSlice(filepath='./user_app_usage.csv',CHUNK_SIZE = 10000000):
    '''
    for slicing file into many small files, default size of small file is 20*CHUNKSIZE
    output file name is
    '''
    reader = pd.read_csv(filepath, iterator=True, names=['uId','appId','duration','times','use_date'])
    
    EOF = True
    i = 0
    s = 0
    start = time.time()
    count = 0
    while EOF:
        try:
            chunk = reader.get_chunk(CHUNK_SIZE)
            i+=1
            gap = time.time()
            s=s+gap-start
            print(i,' time gap:',gap-start,' all time:',s,chunk.shape)
            start = gap
            chunk.to_csv('{0}_{1}.csv'.format(os.path.splitext(os.path.basename(filepath))[0],i//20+1),header = False, mode='a')

        except:
            EOF = False
            print('end of file')
            print(i*CHUNK_SIZE)



In [10]:
# data cleaning:

In [5]:
### clean the date type colums

rd = pd.read_csv('./uau_4.csv',iterator=True, names=['uId','appId','duration','times','use_date'])
chunk = rd.get_chunk(1000000)

dic_date_int = dateToInt(chunk.values[:,4].min(),chunk.values[:,4].max())

start = time.time()

for k,v in dic_date_int.items():
    chunk.replace(k,v,inplace=True) # the reason for use_date replace operation is fast is that the data cluster massively

end = time.time()

print(chunk,end-start)



NameError: name 'pd' is not defined

In [11]:
### clean the appId column
start = time.time()
dic_appId_int = appIdToInt(chunk.values[:,1])

tmp = chunk
# for v in tmp.values:
# #     chunk.replace(k,v,inplace=True) # the speed is very slow
#       print(v)  

end = time.time()

print(chunk,end-start)

NameError: name 'chunk' is not defined

In [166]:
# testing:

rd = pd.read_csv('./uau_4.csv',iterator=True, names=['uId','appId','duration','times','use_date'])
chunk = rd.get_chunk(1000000)

# clean the date type colums
dic_date_int = dateToInt(chunk.values[:,4].min(),chunk.values[:,4].max())

start = time.time()

for k,v in dic_date_int.items():
    chunk.replace(k,v,inplace=True)

end = time.time()

print(chunk,end-start)


               uId      appId  duration  times  use_date
590000000  2345916  a00275200      17.0   10.0        16
590000001  2345916  a00287397      22.0    5.0        10
590000002  2345916  a00274701   11001.0  368.0        23
590000003  2345916  a00289866     678.0   15.0        24
590000004  2345916  a00275200    1217.0   19.0        24
590000005  2345916  a00311680       3.0    1.0         6
590000006  2345916  a00275200     258.0   61.0         0
590000007  2345916  a00336224       6.0    4.0        14
590000008  2345916  a00289826      52.0    4.0        19
590000009  2345916  a00450743     107.0    2.0        18
590000010  2345916  a00287397      77.0    9.0        19
590000011  2345916  a00289866     678.0   15.0        25
590000012  2345916  a00274701    1519.0   89.0        24
590000013  2345916  a00289519     233.0   12.0         0
590000014  2345916  a00311680      22.0    2.0         4
590000015  2345916  a00289826     235.0    8.0        12
590000016  2345916  a00275200  